<a href="https://colab.research.google.com/github/AftabKhalil/Colabs/blob/main/Natural_Language_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Qs 3. Wrangle / change the data so that you can instead build a "positive" emotion vs "negative" emotions classifier, by combining labels. Can you now report the accuracy of a classifer on this data? You may use one of the model architectures from (1) or (2)</h1>

<h1>Installing the dependencies first<h1>

In [ ]:
!pip install datasets==1.3.0
!pip install transformers==4.3.2
!pip install folium==0.2.1
!pip install imgaug==0.2.6
!pip install numpy==1.19.2

<h2>Load dataset</h2>

In [ ]:
from datasets import load_dataset
emotions_dataset = load_dataset('go_emotions', 'simplified')

Reusing dataset go_emotions (/root/.cache/huggingface/datasets/go_emotions/simplified/0.0.0/ef1c18ea192c771555f1e0d638889dd5f1896255782c57c6a0b934d5f94f779e)


<h1>A look at data</h1>
To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
# For simplicity, train model with single label for item.
restricted_dataset = emotions_dataset.filter(lambda x: len(x["labels"]) == 1 and 27 not in x['labels'])
print(len(restricted_dataset['train']))
show_random_elements(restricted_dataset['train'])


23485


<h1>Seperating Positive and Negative labels</h1>

In [ ]:
labels = ["admiration", "amusement", "anger", "annoyance", "approval", "caring", "confusion", "curiosity", "desire", "disappointment", "disapproval",
          "disgust", "embarrassment", "excitement", "fear", "gratitude", "grief", "joy", "love", "nervousness", "optimism", "pride", "realization",
          "relief", "remorse", "sadness", "surprise", "neutral"]
positive = ["admiration", "amusement", "approval", "caring", "curiosity", "desire", "excitement", 
            "gratitude", "joy", "love", "optimism", "pride", "realization", "relief", "surprise", "neutral"]
negative = ["anger", "annoyance", "confusion", "disappointment", "disapproval", "disgust", 
            "embarrassment", "fear", "grief", "nervousness", "remorse", "sadness"]

index_to_labels = {index: label for index, label in enumerate(labels)}
print(index_to_labels)

{0: 'admiration', 1: 'amusement', 2: 'anger', 3: 'annoyance', 4: 'approval', 5: 'caring', 6: 'confusion', 7: 'curiosity', 8: 'desire', 9: 'disappointment', 10: 'disapproval', 11: 'disgust', 12: 'embarrassment', 13: 'excitement', 14: 'fear', 15: 'gratitude', 16: 'grief', 17: 'joy', 18: 'love', 19: 'nervousness', 20: 'optimism', 21: 'pride', 22: 'realization', 23: 'relief', 24: 'remorse', 25: 'sadness', 26: 'surprise', 27: 'neutral'}


<h1>Lets change the labels in data set from 27 classes to 2 claases, i.e Negative and Positive</h1>
And have a look at new data

In [ ]:
import copy

#deep copy the data set
pn_restricted_dataset = copy.deepcopy(restricted_dataset)

#Map all the positive classes to 0 and negative to 1
pn_restricted_dataset['train'] = pn_restricted_dataset['train'].map(lambda x: {'text' : x['text'], 'labels' : [0] if index_to_labels[(x['labels'])[0]] in positive else [1]})
pn_restricted_dataset['validation'] = pn_restricted_dataset['validation'].map(lambda x: {'text' : x['text'], 'labels' : [0] if index_to_labels[(x['labels'])[0]] in positive else [1]})
pn_restricted_dataset['test'] = pn_restricted_dataset['test'].map(lambda x: {'text' : x['text'], 'labels' : [0] if index_to_labels[(x['labels'])[0]] in positive else [1]})

show_random_elements(pn_restricted_dataset['train'])

<h1>Method to define and preprocess the model</h1>

In [ ]:
from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer

def load_model(name):
  global model
  global tokenizer

  model_name = name
  config = AutoConfig.from_pretrained(model_name, num_labels=2)
  model = AutoModelForSequenceClassification.from_config(config=config)
  tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
  encode_dataset()

def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True)

def convert_labels_to_int(example):
    example['labels'] = example['labels'][0]
    return example

def encode_dataset():
  global encoded_dataset
  encoded_dataset = pn_restricted_dataset.map(preprocess_function, batched=True)
  encoded_dataset = encoded_dataset.map(convert_labels_to_int)

## Fine-tuning the model

In [ ]:
metric = datasets.load_metric('accuracy')
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis = 1)
    return metric.compute(predictions = predictions, references = labels)

<h1>Method to train the model</h1>

In [ ]:
import numpy as np
from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments

def runModel(epoch_size, learning_rate):
  print(f'Training model for epoch_size : {epoch_size} and learning_rate : {learning_rate}')
  metric_name = "accuracy"

  args = TrainingArguments(
      "test-emotions",
      evaluation_strategy = "epoch",
      learning_rate = learning_rate,
      per_device_train_batch_size = 32,
      per_device_eval_batch_size = 32,
      num_train_epochs = epoch_size,
      weight_decay = 0.01,
      #We should set load_best_model_at_end to False as we want to deterine the effect of batch sizes
      #So if accuracy decrease after certain batches, it will still give the best model that will
      #not fulfill our purpose
      load_best_model_at_end = True,
      metric_for_best_model = metric_name)

  validation_key = "validation"

  trainer = Trainer(
      model,
      args,
      train_dataset = encoded_dataset["train"],
      eval_dataset = encoded_dataset["validation"],
      tokenizer = tokenizer,
      compute_metrics = compute_metrics)

  trainer.train()
  res = trainer.evaluate()
  return res

<h1>Finally, Load and train the model<h1>

In [ ]:
load_model('distilbert-base-uncased-finetuned-sst-2-english')
res = runModel(2, 2e-4)
print(res)


Training model for epoch_size : 2 and learning_rate : 0.0002


{'eval_loss': 0.4162618815898895, 'eval_accuracy': 0.8200270635994588, 'eval_runtime': 3.2298, 'eval_samples_per_second': 915.234, 'epoch': 2.0}


<h1>Model accuraccy is increases to <b>0.82</b> from 0.57.</h1>

<h1>Method for prediction</h1>

In [ ]:
import torch
def predict(message):
  device = "cuda:0"
  input = tokenizer(message, return_tensors="pt").to(device)
  output = model(**input)
  p = output[0].cpu().data.numpy().argmax()
  return "Positive" if p == 0 else "Negative"

<h1>Predict a postive sentense</h1>

In [ ]:
message = "This is good."
prediction = predict(message)
print(prediction)

Positive


<h1>Predict a negative sentense</h1>

In [ ]:
message = "This is discusting."
prediction = predict(message)
print(prediction)

Negative


<hr>

<h1>Saving the model (in google drive)</h1>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import torch
model_save_name = 'nlp_model_binary.pt'
path = f'/content/gdrive/My Drive/models/{model_save_name}'
model.save_pretrained(path)

<h1>Load saved model</h1>

In [ ]:
loaded_model = model.from_pretrained(path)

<h1>Method to do prediction from loaded model</h1>

In [ ]:
def predict_from_loaded(message):
  device = "cuda:0"
  input = tokenizer(message, return_tensors="pt").to(device)
  loaded_model.to(device)
  output = loaded_model(**input)
  p = output[0].cpu().data.numpy().argmax()

  return "Positive" if p == 0 else "Negative"

<h1>Predict a postive sentense</h1>

In [ ]:
message = "I like it"
prediction = predict_from_loaded(message)
print(prediction)

Positive


<h1>Predict a negative sentense</h1>

In [ ]:
message = "It dont like it"
prediction = predict_from_loaded(message)
print(prediction)

Negative


<h1>End 🐍</h1>